In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F

from netam import framework, models
from epam.torch_common import pick_device, print_parameter_count

/Users/matsen/mambaforge/envs/epam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
all_df = pd.read_csv(shmoof_data_path)

# Here's the fraction of sequences of length more than 410
(all_df["parent"].str.len() > 410).sum() / len(all_df)

0.00403216242498992

In [3]:
train_df, val_df = framework.load_shmoof_dataframes("/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv", val_nickname="51") #, sample_count=5000)


In [4]:
kmer_length = 3
max_length = 410

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Using Metal Performance Shaders
we have 26592 training examples and 22424 validation examples


In [5]:
cat_params = {
    "kernel_size": [9, 11, 13, 15, 17, 19],
    "dropout": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
}
int_params = {
    "embedding_dim": (5, 17),
    "num_filters": (7, 40),
}
float_params = {
}
log_float_params = {
}
# Note that if anything appears below and above, the above gets priority.
fixed_hyperparams = {
    "num_filters": 12,
    "embedding_dim": 18,
    "kernel_size": 13,
    "batch_size": 1024,
    "learning_rate": 0.1,
    "min_learning_rate": 1e-3, # early stopping!
    "dropout": 0.1,
    "l2_regularization_coeff": 1e-6,
    "min_parameter_count": 4096,
    "max_parameter_count": 8192,
}
epochs = 100
optuna_steps = 45

hyper_burrito = framework.HyperBurrito(pick_device(), train_dataset, val_dataset, models.CNNModel,  epochs=epochs)

hyper_burrito.optuna_optimize(optuna_steps, cat_params, int_params, float_params, log_float_params, fixed_hyperparams)

[I 2023-11-28 07:39:02,139] A new study created in memory with name: no-name-77def931-3b1d-4e19-acff-32cd0fd375a8


Using Metal Performance Shaders


Epoch: 100%|██████████| 100/100 [12:03<00:00,  7.24s/it, loss_diff=-3.836e-05, lr=0.1]
[I 2023-11-28 07:51:10,065] Trial 0 finished with value: 0.06193994804472529 and parameters: {'kernel_size': 15, 'dropout': 0.0, 'embedding_dim': 16, 'num_filters': 19}. Best is trial 0 with value: 0.06193994804472529.
[I 2023-11-28 07:51:10,068] Trial 1 finished with value: 1000000000.0 and parameters: {'kernel_size': 13, 'dropout': 0.2, 'embedding_dim': 5, 'num_filters': 25}. Best is trial 0 with value: 0.06193994804472529.
[I 2023-11-28 07:51:10,071] Trial 2 finished with value: 1000000000.0 and parameters: {'kernel_size': 13, 'dropout': 0.0, 'embedding_dim': 6, 'num_filters': 9}. Best is trial 0 with value: 0.06193994804472529.
[I 2023-11-28 07:51:10,073] Trial 3 finished with value: 1000000000.0 and parameters: {'kernel_size': 19, 'dropout': 0.3, 'embedding_dim': 10, 'num_filters': 16}. Best is trial 0 with value: 0.06193994804472529.


Trial rejected. Model has 2001 parameters, not in [4096, 8192]].
Trial rejected. Model has 1111 parameters, not in [4096, 8192]].
Trial rejected. Model has 3723 parameters, not in [4096, 8192]].


Epoch:  30%|███       | 30/100 [03:20<07:46,  6.67s/it, loss_diff=-2.041e-09, lr=3.2e-5] 
[I 2023-11-28 07:54:34,608] Trial 4 finished with value: 0.06571209818906244 and parameters: {'kernel_size': 17, 'dropout': 0.3, 'embedding_dim': 15, 'num_filters': 21}. Best is trial 0 with value: 0.06193994804472529.
Epoch:  43%|████▎     | 43/100 [06:41<08:52,  9.34s/it, loss_diff=1.409e-07, lr=3.2e-5]  
[I 2023-11-28 08:01:22,479] Trial 5 finished with value: 0.06041432957387889 and parameters: {'kernel_size': 19, 'dropout': 0.4, 'embedding_dim': 7, 'num_filters': 33}. Best is trial 5 with value: 0.06041432957387889.
Epoch: 100%|██████████| 100/100 [12:32<00:00,  7.52s/it, loss_diff=-3.141e-05, lr=0.1]
[I 2023-11-28 08:13:59,588] Trial 6 finished with value: 0.06359826900195191 and parameters: {'kernel_size': 17, 'dropout': 0.1, 'embedding_dim': 12, 'num_filters': 23}. Best is trial 5 with value: 0.06041432957387889.
[I 2023-11-28 08:13:59,594] Trial 7 finished with value: 1000000000.0 and par

...

Trial rejected. Model has 3927 parameters, not in [4096, 8192]].


Epoch: 100%|██████████| 100/100 [11:31<00:00,  6.91s/it, loss_diff=-2.306e-05, lr=0.1]
[I 2023-11-28 11:40:29,492] Trial 35 finished with value: 0.06480948779967229 and parameters: {'kernel_size': 15, 'dropout': 0.5, 'embedding_dim': 17, 'num_filters': 20}. Best is trial 8 with value: 0.060356456682729244.
Epoch:  44%|████▍     | 44/100 [05:54<07:30,  8.05s/it, loss_diff=1.504e-06, lr=3.2e-5]  
[I 2023-11-28 11:46:29,059] Trial 36 finished with value: 0.060334884378523475 and parameters: {'kernel_size': 15, 'dropout': 0.0, 'embedding_dim': 14, 'num_filters': 25}. Best is trial 36 with value: 0.060334884378523475.
Epoch: 100%|██████████| 100/100 [12:24<00:00,  7.44s/it, loss_diff=-4.913e-05, lr=0.1]
[I 2023-11-28 11:58:59,294] Trial 37 finished with value: 0.06310477301005249 and parameters: {'kernel_size': 13, 'dropout': 0.0, 'embedding_dim': 14, 'num_filters': 26}. Best is trial 36 with value: 0.060334884378523475.
Epoch: 100%|██████████| 100/100 [13:55<00:00,  8.36s/it, loss_diff=-3.

Trial rejected. Model has 2993 parameters, not in [4096, 8192]].


Epoch: 100%|██████████| 100/100 [12:37<00:00,  7.58s/it, loss_diff=-6.206e-05, lr=0.1]
[I 2023-11-28 12:39:09,050] Trial 41 finished with value: 0.06269341754942037 and parameters: {'kernel_size': 15, 'dropout': 0.0, 'embedding_dim': 16, 'num_filters': 18}. Best is trial 36 with value: 0.060334884378523475.
Epoch: 100%|██████████| 100/100 [13:58<00:00,  8.39s/it, loss_diff=-3.988e-05, lr=0.1]
[I 2023-11-28 12:53:13,507] Trial 42 finished with value: 0.06439086422855796 and parameters: {'kernel_size': 15, 'dropout': 0.5, 'embedding_dim': 15, 'num_filters': 23}. Best is trial 36 with value: 0.060334884378523475.
Epoch: 100%|██████████| 100/100 [13:19<00:00,  7.99s/it, loss_diff=-5.412e-06, lr=0.02]
[I 2023-11-28 13:06:37,862] Trial 43 finished with value: 0.06523945325964951 and parameters: {'kernel_size': 15, 'dropout': 0.2, 'embedding_dim': 14, 'num_filters': 17}. Best is trial 36 with value: 0.060334884378523475.
Epoch:  29%|██▉       | 29/100 [03:54<09:33,  8.07s/it, loss_diff=-1.953

Best Hyperparameters: {'kernel_size': 15, 'dropout': 0.0, 'embedding_dim': 14, 'num_filters': 25}
Best Validation Loss: 0.060334884378523475
